## import libraries
***

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import time
import json
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout,Bidirectional
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
import mediapipe as mp

***

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)
tf.config.list_physical_devices('GPU')
# tf.device('/gpu:0')

***
## MobileNetV2_____START##########################

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
IMG_SHAPE = (160, 160, 3)
MobileNetV2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
MobileNetV2.summary()

In [ ]:
nb_layers = len(MobileNetV2.layers)
print(MobileNetV2.layers[nb_layers - 2].name)
print(MobileNetV2.layers[nb_layers - 1].name)

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = MobileNetV2(x)

x =tfl.GlobalAveragePooling2D()(x)
model = tf.keras.Model(inputs,x)

In [ ]:
model.summary()


***

***
## load json into dataFrame_____Start###############

In [2]:
with open('F://aaaaaaaaaaa//ml_advanced_ng//Certificates//WLASL_v0.3.json', 'r') as f:
  jdata = json.load(f)

In [16]:
for i in range(len(jdata[199]['instances'])):
    
    print(jdata[199]['instances'][i]['video_id'])

14780
69291
14797
14799
14800
14801
65444
14792
67553
14802
14803
14793
14794
14795
14796


In [ ]:
# jdata = jdata[50:128]

vidsIDS={}
for i in range(10):
    vidsIDS[jdata[i]["gloss"]] = []

# print(len(jdata),len(vidsIDS))
len(vidsIDS.keys())
vidsIDS.keys()

In [ ]:
wordlen = len(vidsIDS)
k=0
for i in vidsIDS.keys():
    wordVidsLen = len(jdata[k]['instances'])
    for j in range(wordVidsLen):
        vidsIDS[i].append(jdata[k]['instances'][j]['video_id'])        
    k+=1
k

In [ ]:
del jdata

In [ ]:
for i in vidsIDS.keys():
    print(len(vidsIDS[i]))

In [ ]:
cols = list(vidsIDS.keys())
data = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in vidsIDS.items() ]),columns =cols)

In [ ]:
data = data.astype("str")

In [ ]:
for i in cols:
    data.loc[:,i] = data.loc[:,i].apply(lambda x:"F://aaaaaaaaaaa//ml_advanced_ng//Certificates//videos//"+x+".mp4")

In [ ]:
data.iloc[1,2]

In [ ]:
data.iloc[2,0]
data.shape


***

***
## holistic________START###########

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    return results

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) 

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


***

***
## video Processing & saving .npy files________START###########

In [ ]:
def videoProc(path=data.iloc[2,1]):
    cap = cv2.VideoCapture(path)
    success, frame = cap.read()
    fno = 0
    framecount = 60
    v = None
    if success:
        # img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # img = cv2.resize(img, (160,160), interpolation = cv2.INTER_AREA)
        # v = model.predict_step(np.array([img]))
        # height = frame.shape[0]
        # width = frame.shape[1]
        # print(width,height)
        
        img = frame
        v =  np.reshape(extract_keypoints(mediapipe_detection(frame, holistic)), (1,-1))
        # print(v.shape)
        fno = 1
        while True:
            # Show to screen
            # cv2.imshow('OpenCV Feed', frame):
            if fno >=framecount:
                break 
            success, frame = cap.read()

            if success:
                # img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # v = np.concatenate((v, model.predict_step(np.array([img]))), axis=0) 
                
                v = np.concatenate((v,np.reshape(extract_keypoints(mediapipe_detection(frame, holistic)), (1,-1))), axis=0) 
                fno += 1
            else:
                break
            # Break 

    cap.release()
    cv2.destroyAllWindows()
    if fno<framecount and fno>0:
        tmp = np.zeros((framecount-fno,1662)) 
        v = np.concatenate((v, tmp), axis=0) 

    return fno,v

In [ ]:
fno,vidarr = videoProc(path=data.iloc[1,5])
vidarr[0]

In [ ]:
len(cols)

In [ ]:
vidno = 0
_i=0
for w in cols:
    instantsLen = len(data.loc[:,w])
    data.loc[:,w] = data.loc[:,w].apply(str)
    print(w+"___no#"+str(_i))
    _i+=1
    for i in range(instantsLen):
        s = str(data.loc[i,w])
        if s.split("//")[-1] == "nan.mp4":
            # print(s)
            break
        fno,vidarr = videoProc(path=data.loc[i,w])

        # print(fno)
        if fno == 0:
            continue

        vidno += 1
        # np.save('.//vidsNumpy//'+w+'__'+str(i)+'.npy', vidarr)        
        np.save('.//vnh//'+w+'__'+str(i)+'.npy', vidarr)

In [ ]:
vidno


***

***
## use tf.data.dataset to create data pipe line_______START###########

In [ ]:
def read_npy_file(item):
    
    data = np.load(item.numpy().decode())
    return data
#, label_map[item.numpy().decode().split('\\')[-1].split('__')[0]]

In [ ]:
file_list = os.listdir("./vnh")
random.shuffle(file_list)


In [ ]:
formLableToNum = {label:num for num, label in enumerate(cols)}
formNumTOLabel = {num:label for num, label in enumerate(cols)}
formLableToNum

In [ ]:
lables = [f.split('__')[0] for f in file_list]
ln = list(np.unique(lables))
len(ln)
lables[0:4]

In [ ]:
lables = [formLableToNum[f.split('__')[0]] for f in file_list]


In [ ]:
label_data = tf.data.Dataset.from_tensor_slices(lables)

In [ ]:
file_list = [".\\vnh\\"+f for f in file_list]
dataset = tf.data.Dataset.from_tensor_slices(file_list)
len(dataset)

In [ ]:
dataset = dataset.map(lambda item:tf.py_function(read_npy_file, [item], [tf.float32]))
dataset = tf.data.Dataset.zip((dataset, label_data))

In [ ]:
train_size = int(len(dataset)*0.8)

train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)

In [ ]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:

for n,l in dataset.take(5):
    print(n[0].shape)

***

***
## create lstm model______START################

In [ ]:
model_lstm_1 = Sequential([

tf.keras.Input(shape=(40,1662),),

Bidirectional(LSTM(128, return_sequences=True, activation='tanh'),input_shape=(40,1662)), 
Dropout(0.1, noise_shape = (1,80, 1)),
    
Bidirectional(LSTM(128, return_sequences=True, activation='tanh'),input_shape=(40,1662)), 
Dropout(0.1, noise_shape =  (1,80, 1)),

LSTM(128, return_sequences=False, activation='tanh', recurrent_dropout=0),
BatchNormalization(axis=-1,center=True,scale=True,),
Dropout(0.1,),

      
# LSTM(128, return_sequences=False, activation='relu', recurrent_dropout=0.2),
# BatchNormalization(axis=-1,center=True,scale=True,),
# Dropout(0.1,),
    
Dense(128, activation='linear',kernel_regularizer=tf.keras.regularizers.L2(1)),
BatchNormalization(axis=-1,center=True,scale=True,),
ReLU(),
    
Dense(128, activation='linear',kernel_regularizer=tf.keras.regularizers.L1(0.1)),
BatchNormalization(axis=-1,center=True,scale=True,),
ReLU(),
    
Dense(30, activation='softmax')
])


In [ ]:
model_lstm_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,)
,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["accuracy"])

In [ ]:
model_lstm_1.fit(train_ds.batch(32), epochs=256, )

In [ ]:
model_lstm_1.summary()

***

***
## test_______START############

In [ ]:
train_ds_l =[]
train_ds_np = []
for i,j in train_ds.take(1000):
    train_ds_l.append(j.numpy())
    train_ds_np.append(i[0].numpy())
    
train_ds_np = np.array(train_ds_np)
train_ds_l = np.array(train_ds_l)
train_ds_np.shape

In [ ]:
p = model_lstm_1.predict(train_ds_np)
a = np.sum((np.argmax(p,axis=1)==train_ds_l))/ p.shape[0]
print(np.argmax(p[51]),np.argmax(train_ds_l[51]),a)
p.shape

In [ ]:
test_ds_l =[]
test_ds_np = []
for i,j in test_ds:
    test_ds_l.append(j.numpy())
    test_ds_np.append(i[0].numpy())
    
test_ds_np = np.array(test_ds_np)
test_ds_l = np.array(test_ds_l)
test_ds_l.shape



In [ ]:
p = model_lstm_1.predict(test_ds_np)
a = np.sum((np.argmax(p,axis=-1)==test_ds_l))/ p.shape[0]
print(np.argmax(p[2]),np.argmax(test_ds_l[2]),a)
